In [ ]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

import xgboost as xgb
from sklearn.svm import SVC
import lightgbm as lgb

from sklearn.metrics import f1_score,confusion_matrix,roc_auc_score

from tqdm import tqdm_notebook

import warnings
warnings.filterwarnings("ignore")

In [ ]:
# test_output = []
# for i in tqdm_notebook(range(3)):
#     train_data = np.load('../input/melonoma-features-tpu/'+str(i)+'_train_extracted.npy')
#     train_label = np.load('../input/melonoma-features-tpu/'+str(i)+'_train_label_extracted.npy')
#     valid_data = np.load('../input/melonoma-features-tpu/'+str(i)+'_valid_extracted.npy')
#     valid_label = np.load('../input/melonoma-features-tpu/'+str(i)+'_valid_label_extracted.npy')
    
#     test_data = np.load('../input/melonoma-features-tpu/'+str(i)+'_test_extracted.npy')
    
#     model = SVC(kernel='linear',probability=True)
#     model.fit(train_data,train_label)
    
#     valid_pred = model.predict_proba(valid_data)
# #     print('valid_auc: '+str(roc_auc_score(valid_label,np.max(valid_pred,axis=1))))\
#     print('valid_auc: '+str(roc_auc_score(valid_label,valid_pred[:,1]>0.5)))
    
#     test_pred = model.predict_proba(test_data)
# #     np.save(str(i)+'_test_prediction_b3_model.npy',np.max(test_pred,axis=1))
# #     np.save(str(i)+'_test_prediction_b3_model.npy',test_pred)
# #     test_output.append(np.max(test_pred,axis=1))
#     test_output.append(test_pred[:,1])
#     print('test_: '+str((test_pred[:,1]>0.5).sum()))


In [ ]:
test_output = []
for i in tqdm_notebook(range(3)):
    train_data = np.load('../input/melonoma-features-tpu/'+str(i)+'_train_extracted.npy')
    train_label = np.load('../input/melonoma-features-tpu/'+str(i)+'_train_label_extracted.npy')
    valid_data = np.load('../input/melonoma-features-tpu/'+str(i)+'_valid_extracted.npy')
    valid_label = np.load('../input/melonoma-features-tpu/'+str(i)+'_valid_label_extracted.npy')
    
    test_data = np.load('../input/melonoma-features-tpu/'+str(i)+'_test_extracted.npy')
    
    model = lgb.LGBMClassifier(objective='binary',is_unbalance=True)
    model.fit(train_data,train_label)
    
    valid_pred = model.predict_proba(valid_data)
#     print('valid_auc: '+str(roc_auc_score(valid_label,np.max(valid_pred,axis=1))))\
    print('valid_auc: '+str(roc_auc_score(valid_label,valid_pred[:,1]>0.5)))
    
    test_pred = model.predict_proba(test_data)
#     np.save(str(i)+'_test_prediction_b3_model.npy',np.max(test_pred,axis=1))
#     np.save(str(i)+'_test_prediction_b3_model.npy',test_pred)
#     test_output.append(np.max(test_pred,axis=1))
    test_output.append(test_pred[:,1])
    print('test_: '+str((test_pred[:,1]>0.5).sum()))


In [ ]:
test_output = np.array(test_output)
test_output.shape

In [ ]:
test_output_avg = np.average(test_output,axis=0)
(test_output_avg>0.5).sum()

In [ ]:
confusion_matrix(valid_label,valid_pred)

In [ ]:
pred_pro = model.predict_proba(valid_data)

In [ ]:
sns.distplot(pred_pro[:,0])
sns.distplot(pred_pro[:,1])
sns.distplot(np.max(pred_pro,axis=1))
# sns.distplot(np.argmax(pred_pro,axis=1))
plt.show()

In [ ]:
(np.max(pred_pro,axis=1)>0.5).sum()

In [ ]:
(pred_pro[:,1]>0.5).sum()

In [ ]:
roc_auc_score(valid_label,pred_pro[:,1])

In [ ]:
roc_auc_score(valid_label,np.max(pred_pro,axis=1))

In [ ]:
pred_pro = model.predict_proba(test_data)
pred_pro.shape

In [ ]:
# sns.distplot(pred_pro[:,0])
sns.distplot(pred_pro[:,1])
# sns.distplot(np.max(pred_pro,axis=1))
# sns.distplot(np.argmax(pred_pro,axis=1))
plt.show()

In [ ]:
np.unique(pred_pro[:,0]>0.5,return_counts=True)

In [ ]:
np.unique(pred_pro[:,1]>0.3,return_counts=True)

In [ ]:
test_output = np.sum(np.array(test_output),axis=0)
test_output.shape

In [ ]:
test_df = pd.read_csv('../input/siim-isic-melanoma-classification/sample_submission.csv')
test_df.shape

In [ ]:
np.unique(test_output_avg>0.5,return_counts=True)

In [ ]:
test_df['target'] = test_output_avg
# test_df['target_prob'] = pred_pro[:,1]
test_df.head()

In [ ]:
test_df.to_csv('lgbm_tpu_model_prob.csv',index=False)